# Santos Dumont (SD) - Numba CPU

# Abordagem PAD no ambiente Python para um caso de teste de computação científica

Exemplo mostrando abordagen de Processamento de Alto Desempenho (PAD ou HPC) no ambiente Python, para a solução de um caso de teste de computação científica, usando o supercomputador Santos Dumont (SD). Para este exemplo foi selecionado o compilador Numba rodando em CPU e em GPU, usando os nós B710 e Sequana.

## Caso de teste: estêncil de cinco pontos

(versão sequencial)

O caso de teste adotado é um conhecido problema de transferência de calor sobre uma superfície finita, modelado pela equação diferencial parcial de Poisson, que modela a distribuição de temperatura normalizada sobre a superfície ao longo de uma série de iterações que compõem a simulação. A equação de Poisson discretizada 2D com um estêncil de 5 pontos pode ser expressa por:

$$
\frac{\partial^2 U}{\partial x^2} +
\frac{\partial^2 U}{\partial y^2} \approx
\frac{U_{i+1,j}+U_{i,j+1}-4U_{i,j}+U_{i-1,j}+U_{i,j-1}}{h^2}
$$

Comumente empregada para soluções numéricas, esta equação é discretizada em uma grade finita e resolvida por um método de diferenças finitas. O algoritmo específico requer o cálculo de um estêncil de 5 pontos sobre a grade de domínio 2D para atualizar as temperaturas a cada etapa de tempo. Para este problema, são assumidos um campo de temperatura uniforme inicial com valor zero sobre a superfície, e condições adiabáticas ou de contorno de Dirichlet. O estêncil de 5 pontos consiste em atualizar um ponto da grade fazendo a média das temperaturas de si mesmo com as temperaturas de seus quatro vizinhos, esquerda-direita e cima-baixo na grade. O campo de temperatura $ U $ é definido sobre uma grade discreta $ (x, y) $ com resoluções espaciais $ \Delta x = \Delta y = h $. Assim, a discretização mapeia as coordenadas cartesianas reais $ (x, y) $ para uma grade discreta $ (i, j) $, com $ U_ {x, y} = U_ {i, j} \,, \, U_ {x + h, y} = U_ {i + 1, j} \, $ para a dimensão $ x $, e analogamente para a dimensão $ y $. A grade é representada pela figura:

![](img/grade.png)

Três fontes de calor de taxa constante foram colocadas em pontos de grade localizados, e cada uma introduz uma quantidade de calor unitária a cada passo de tempo. A simulação de transferência de calor é modelada por um número finito de etapas de tempo, sendo todos os pontos da grade atualizados a cada etapa. A distribuição da temperatura será determinada pelas fontes de calor e pelas condições de contorno de Dirichlet, o que implica em temperatura zero nos pontos de fronteira da grade. O resultado final é representado pela figura:

![](img/fontes.png)

---

# Implementação

A parte computacionalmente intensiva é a atualização dos pontos da grade (atualização da matriz) utilizando a equação (cálculo do estêncil), onde dois laços de repetição são utilizados, um para cada dimensão.

Em cada iteração, após a atualização da grade uma unidade de calor é inserida em três pontos de inserção de calor.

Duas matrizes são utilizadas alternadamente nas iterações pois o cálculo do estêncil depende de pontos de células vizinhas que não podem ser sobrepostos. Desta forma, uma "nova" matriz de resultados é utilizada para armazenar os cálculos feitos a partir da matriz "antiga" que contém os dados. Quando a matriz "nova" está finalizada, a matriz "antiga" pode então ser descartada e utilizada para armazenar os resultados da próxima iteração, e desta forma as duas matrizes ficam alternando suas funções durante as iterações.

Ao término das iterações, é feita uma soma de todos os pontos da grade, para obter o total de calor.

Nó B710:

In [1]:
! lscpu | head -n 15 | grep "Model \|CPU(s):\|Thre\|Core\|NUMA\|MHz"

CPU(s):                24
Thread(s) per core:    1
Core(s) per socket:    12
NUMA node(s):          2
Model name:            Intel(R) Xeon(R) CPU E5-2695 v2 @ 2.40GHz
CPU MHz:               2435.302


Intel Turbo Boost, ativo = 0 (afeta as medições de velocidade no nó de login):

In [2]:
! cat /sys/devices/system/cpu/intel_pstate/no_turbo

0


Hyperthreading, ativo = 1 :

In [5]:
! cat /sys/devices/system/cpu/smt/active

0


## Fortran 90 sequencial

O código F90 pode ser desenvolvindo de várias maneiras, como por exemplo, usando uma [IDE para F90](https://cbfortran.sourceforge.io/), usando um [Kernel para o Jupyter Notebook (JN)](https://gitlab.com/lfortran/fortran_kernel), ou então usando uma célula do JN como um editor de texto comum para criar/gravar um arquivo no disco, como mostrado a seguir:

In [1]:
%%writefile heat_seq.f90

! computationally intensive core
subroutine kernel(anew, aold, sizeStart, sizeEnd)
    double precision, intent(out) :: anew(:,:)
    double precision, intent(in)  :: aold(:,:)
    integer, intent(in) :: sizeStart, sizeEnd
    integer             :: i, j

    do j = sizeStart, sizeEnd
        do i = sizeStart, sizeEnd
            anew(i,j)= aold(i,j)/2.0  &
                +(aold(i-1,j)+aold(i+1,j)+aold(i,j-1)+aold(i,j+1))/8.0
        enddo
    enddo   
end subroutine

! main routine
program stencil
    implicit none
    interface
        subroutine kernel(anew, aold, sizeStart, sizeEnd)
            double precision, intent(out) :: anew(:,:)
            double precision, intent(in)  :: aold(:,:)
            integer, intent(in) :: sizeStart, sizeEnd
        end subroutine
    end interface
    
    ! parameters for calculation
    integer             :: n=2400     ! n x n grid
    integer             :: energy=1   ! energy to be injected per iteration
    integer             :: niters=250 ! number of iterations

    ! other variables
    integer, parameter  :: nsources=3
    integer, dimension(3, 2)        :: sources
    double precision, allocatable   :: aold(:,:), anew(:,:)
    integer             :: iters, i, j, size, sizeStart, sizeEnd
    double precision    :: heat=0.0           
    
    size = n + 2
    sizeStart = 2
    sizeEnd = n + 1

    allocate(aold(size, size))
    allocate(anew(size, size))
    aold = 0.0
    anew = 0.0

    sources(1,:) = (/ n/2,   n/2   /)
    sources(2,:) = (/ n/3,   n/3   /)
    sources(3,:) = (/ n*4/5, n*8/9 /)

    do iters = 1, niters, 2
        
        ! odd iteration: anew <- stencil(aold)
            
        ! computationally intensive core
        call kernel(anew, aold, sizeStart, sizeEnd)
        
        ! three-point energy insertion
        do i = 1, nsources
            anew(sources(i,1)+1, sources(i,2)+1) =  &
                anew(sources(i,1)+1, sources(i,2)+1) + energy
        enddo

        ! even iteration: aold <- stencil(anew)
           
        ! computationally intensive core
        call kernel(aold, anew, sizeStart, sizeEnd)

        ! three-point energy insertion       
        do i = 1, nsources
            aold(sources(i,1)+1, sources(i,2)+1) =  &
                aold(sources(i,1)+1, sources(i,2)+1) + energy
        enddo

    enddo

    ! sum of grid points to get total heat
    heat = 0.0
    do j = sizeStart, sizeEnd
        do i = sizeStart, sizeEnd
            heat = heat + aold(i,j)
        end do
    end do

    ! show de result
    write(*, "('Heat: ' f0.4)") heat

    deallocate(aold)
    deallocate(anew)

end

Writing heat_seq.f90


> Nota: apesar de funcionar, talvez não seja a melhor forma de utilizar o Jupyter, pois deixa de usar os recursos de documentação presentes.

Na célula a seguir, o ponto de exclamação significa que o comando será executado em uma sessão do shell do sitema operacional:

In [2]:
! gfortran --version

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-36)
Copyright (C) 2015 Free Software Foundation, Inc.

GNU Fortran comes with NO WARRANTY, to the extent permitted by law.
You may redistribute copies of GNU Fortran
under the terms of the GNU General Public License.
For more information about these matters, see the file named COPYING



Compilação usando a flag de otimização `-O3`:

In [3]:
! time gfortran  -O3  -o heat_seq  heat_seq.f90


real	0m0.561s
user	0m0.075s
sys	0m0.032s


Execução do programa:

In [4]:
! time ./heat_seq

Heat: 750.0000

real	0m3.121s
user	0m2.965s
sys	0m0.082s


> Nota: o nó de login não deve ser utilizado para rodar os programas finais; utilizá-lo apenas para compilar, fazer pequenos testes curtos, e gerenciar a fila de execução usando o Slurm.

# Python sequencial (sem Numba)

Versão do Python utilizado:

In [5]:
! python --version

Python 3.8.5


`numpy` é a principal biblioteca para computação científica:

In [6]:
import numpy as np

Os principais parâmetros são:

* `n`: o tamanho da grade (n x n)
* `energy`: a unidade de energia a ser inserida em três pontos em cada iteração
* `niters`: a quantidade de iterações

In [25]:
n      = 2400
energy = 1.0
niters = 250

Outras variáveis e arrays utilidados:
* `anew` e `aold`: matrizes (arrays) para armazenar a grade, e também a zona de fronteira da grade
* `sources`: três pontos de inserção de energia

In [26]:
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

Neste exemplo, para a implementação da equação foram utilizados algoritmos disponíveis na literatura, como os descritos por [Zhu et al.](https://ieeexplore.ieee.org/document/7152597) e [Balaji](https://www.mcs.anl.gov/~thakur/sc17-mpi-tutorial/):

$\mathbf{for} \ \ i \ \leftarrow \ 1 \ \ \mathbf{to} \ \ (matrix \ size \ x) \ \ \mathbf{do}$

$\quad \mathbf{for} \ \ j \ \leftarrow \ 1 \ \ \mathbf{to} \ \ (matrix \ size \ y) \ \ \mathbf{do}$

$\qquad anew[i,j]= \frac{aold[i,j]}{2.0}+\frac{aold[i-1,j]+aold[i+1,j]+aold[i,j-1]+aold[i,j+1]}{8.0}$

Na célula a seguir é possível visualizar a equação e os laços de repetição que atualizam a grade. O laços estão embutidos na sintaxe `:` (dois pontos) dos índices da matriz 
(array numpy). Índices negativos significam o final da matriz. A matriz `aold` contém os dados, e a matriz `anew` armazena os resultados dos cálculos. É o trecho de código de computação intensiva, e o fato de ter sido colocado em uma função será útil nos próximos exemplos:

In [27]:
def kernel(anew, aold) :
    anew[1:-1,1:-1] = (aold[1:-1,1:-1]/2.0
        +(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2])/8.0)

A cada passo do laço de repetição principal, são realizas duas chamadas à função `kernel` para atualizar a grade, desta forma reduzindo pela metade a quantidade de repetições necessárias no laço. A linha de código

```python
anew[sources[:, 0], sources[:, 1]] += energy
```

utiliza a sintaxe "`:` (dois pontos) nos índices dos arrays para indicar laço de repetição embutido, que é usado para inserir (somar) os três pontos de energia na grade. `t2` e `t3` são usados para medir o tempo de execução. `anew` e `aold` são as matrizes que ficam alternando nas funções de armazenar o resultado, e de manter os dados durante o cálculo do estêncil. `%%timeit` é um comando o JN que mede o tempo de execução, neste caso executando uma vez a célula (`-r1`), e a cada vez que a célula é executada realiza um laço de repetição (`-n1`):

In [42]:
%%timeit -r1 -n1
for _ in range(0, niters, 2):
    kernel(anew, aold)
    anew[sources[:, 0], sources[:, 1]] += energy
    kernel(aold, anew)
    aold[sources[:, 0], sources[:, 1]] += energy

33.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Após as iterações, é feita a soma das energias em cada ponto da grade (soma de todos os elementos da matriz), para obter o total de calor:

In [29]:
f"Heat: {np.sum(aold[1:-1, 1:-1]):.4f}"

'Heat: 750.0000'

`%%prun` usa o Python profiler e mostra os tempos com mais detalhes:

In [30]:
%%prun -l 5 -s cumulative
for _ in range(0, niters, 2):
    kernel(anew, aold)
    anew[sources[:, 0], sources[:, 1]] += energy
    kernel(aold, anew)
    aold[sources[:, 0], sources[:, 1]] += energy

         253 function calls in 35.546 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   35.546   35.546 {built-in method builtins.exec}
        1    0.020    0.020   35.546   35.546 <string>:1(<module>)
      250   35.526    0.142   35.526    0.142 <ipython-input-27-c8dd1f9fd6f9>:1(kernel)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

---

Fazendo um teste com `n = 8`, apenas para visualizar a grade final:

In [16]:
np.set_printoptions(precision=1, floatmode='fixed')
aold

array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.8, 1.5, 1.3, 1.0, 0.8, 0.6, 0.4, 0.2, 0.0],
       [0.0, 1.5, 4.0, 2.6, 2.1, 1.6, 1.2, 0.8, 0.4, 0.0],
       [0.0, 1.3, 2.5, 2.8, 3.1, 2.4, 1.8, 1.2, 0.6, 0.0],
       [0.0, 1.0, 2.1, 3.1, 5.1, 3.1, 2.3, 1.7, 0.9, 0.0],
       [0.0, 0.8, 1.6, 2.3, 3.1, 2.7, 2.5, 2.4, 1.2, 0.0],
       [0.0, 0.6, 1.1, 1.6, 2.1, 2.2, 2.6, 4.1, 1.6, 0.0],
       [0.0, 0.4, 0.7, 1.0, 1.3, 1.5, 1.7, 1.9, 1.0, 0.0],
       [0.0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.8, 0.4, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

---

# Numba sequencial (1 thread)

Numba reconhece apenas um subconjunto de Python e Numpy, e é usado geralmente apenas nos trechos que exigem intensidade computacional. Desta forma, a parte do código que não exige esforço computacional, é executada de forma interpretada por Python padrão.

Para o exemplo a seguir, adicionamos medição de tempo e também um decorador na função `kernel` para que Numba a compile em tempo de execução, JIT.

Com relação ao código anterior, a biblioteca `numba` (compilador Numba) é adicionada:

In [44]:
import numpy as np
from numba import njit, set_num_threads, get_num_threads, threading_layer

Este trecho é o mesmo do exemplo anterior:

In [45]:
# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations
# initialize the data arrays
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
# initialize three heat sources
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

A função `kernel`recebe um decorador para indicar para Numba que a função deve ser compilada em tempo de execução, JIT. `float64` são os tipos dos parâmetros, `fastmat` aumenta a velocidade dos cálculos em detrimento da precisão, `parallel` indica a utilização de otimizações como vetorização, e `nogil` desliga o GIL (*global interpreter lock*).

In [46]:
# computationally intensive core
@njit('(float64[:,:],float64[:,:])', parallel=True, fastmath=True, nogil=True)
def kernel(anew, aold) :
    anew[1:-1,1:-1] = (aold[1:-1,1:-1]/2.0
        +(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2])/8.0)

t0, t1, etc., são usados para medição de tempo, `threading_layer` mostra qual biblioteca está sendo usada (OpenMP, Intel TBB, ou biblioteca própria), e `get_num_threads` mostra quantos threads estão sendo usados na execução: se > 1 significa que estão sendo utilizados recursos de processamento paralelo usando threads:

In [47]:
%%prun -l 5 -s cumulative
# main routine
set_num_threads(1)       # set the number of threads
for _ in range(0, niters, 2) :
    kernel(anew, aold)
    anew[sources[:, 0], sources[:, 1]] += energy
    kernel(aold, anew)
    aold[sources[:, 0], sources[:, 1]] += energy
heat = np.sum(aold)      # system total heat

         463 function calls (445 primitive calls) in 3.263 seconds

   Ordered by: cumulative time
   List reduced from 68 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.263    3.263 {built-in method builtins.exec}
        1    0.030    0.030    3.263    3.263 <string>:2(<module>)
      250    3.229    0.013    3.229    0.013 <ipython-input-46-e40ce3ed2a99>:2(kernel)
        1    0.000    0.000    0.004    0.004 <__array_function__ internals>:2(sum)
        1    0.000    0.000    0.004    0.004 {built-in method numpy.core._multiarray_umath.implement_array_function}

In [48]:
# show the result
print(f"Heat: {heat:.4f}", end=" | ")
print(f"Threading layer: {threading_layer()}", end=" | ")
print(f"Thread count: {get_num_threads()}")

Heat: 750.0000 | Threading layer: tbb | Thread count: 1


## 4 threads

> Notas:
> * o *hyperthreading* e o *turboboost* devem estar desligados, caso contrário não é possível observar claramente o aumento de velocidade com o aumento do número de threads
> * A execução usando threads (OpenMP, etc.) geralmente só é possível em um único nó. Para utilizar vários nós de computação, geralmente se usa a biblioteca de troca de mensagens (MPI) que utiliza processos ao invés de threads. Também é possível combinar OpenMP e MPI, embora torne o código mais complexo.
> * Threads utiliza o modelo de memória compartilhada, e processos utiliza o modelo de memória com espaços de endereçamento separados.

In [49]:
%%prun -l 5 -s cumulative
import numpy as np
from numba import njit, set_num_threads, get_num_threads, threading_layer

# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations
# initialize the data arrays
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
# initialize three heat sources
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# computationally intensive core
@njit('(float64[:,:],float64[:,:])', parallel=True, fastmath=True, nogil=True)
def kernel(anew, aold) :
    anew[1:-1,1:-1] = (aold[1:-1,1:-1]/2.0
        +(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2])/8.0)

# main routine
set_num_threads(4)       # set the number of threads
for _ in range(0, niters, 2) :
    kernel(anew, aold)
    anew[sources[:, 0], sources[:, 1]] += energy
    kernel(aold, anew)
    aold[sources[:, 0], sources[:, 1]] += energy
heat = np.sum(aold)    # system total heat

# show the result
print(f"Heat: {heat:.4f}", end=" | ")
print(f"Threading layer: {threading_layer()}", end=" | ")
print(f"Thread count: {get_num_threads()}")

Heat: 750.0000 | Threading layer: tbb | Thread count: 4
 

         2015184 function calls (1808402 primitive calls) in 2.508 seconds

   Ordered by: cumulative time
   List reduced from 2348 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.001    0.000    2.535    2.535 {built-in method builtins.exec}
        1    0.000    0.000    1.565    1.565 decorators.py:188(wrapper)
     48/1    0.000    0.000    1.564    1.564 compiler_lock.py:29(_acquire_compile_lock)
        1    0.000    0.000    1.564    1.564 dispatcher.py:795(compile)
        1    0.000    0.000    1.564    1.564 dispatcher.py:77(compile)

## 24 threads

In [50]:
%%prun -l 5 -s cumulative
import numpy as np
from numba import njit, set_num_threads, get_num_threads, threading_layer

# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations
# initialize the data arrays
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
# initialize three heat sources
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# computationally intensive core
@njit('(float64[:,:],float64[:,:])', parallel=True, fastmath=True, nogil=True)
def kernel(anew, aold) :
    anew[1:-1,1:-1] = (aold[1:-1,1:-1]/2.0
        +(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2])/8.0)

# main routine
set_num_threads(24)       # set the number of threads
for _ in range(0, niters, 2) :
    kernel(anew, aold)
    anew[sources[:, 0], sources[:, 1]] += energy
    kernel(aold, anew)
    aold[sources[:, 0], sources[:, 1]] += energy
heat = np.sum(aold)    # system total heat

# show the result
print(f"Heat: {heat:.4f}", end=" | ")
print(f"Threading layer: {threading_layer()}", end=" | ")
print(f"Thread count: {get_num_threads()}")

Heat: 750.0000 | Threading layer: tbb | Thread count: 24
 

         2014678 function calls (1807939 primitive calls) in 2.371 seconds

   Ordered by: cumulative time
   List reduced from 2337 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.001    0.000    2.408    2.408 {built-in method builtins.exec}
        1    0.000    0.000    1.759    1.759 decorators.py:188(wrapper)
     48/1    0.000    0.000    1.759    1.759 compiler_lock.py:29(_acquire_compile_lock)
        1    0.000    0.000    1.759    1.759 dispatcher.py:795(compile)
        1    0.000    0.000    1.759    1.759 dispatcher.py:77(compile)